# Imports

In [186]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import soundfile as sf
from scipy.fft import rfft, rfftfreq
from sklearn.metrics import accuracy_score
import librosa
import resampy
import librosa.display
import IPython.display as ipd
import os
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from pydub import AudioSegment
import os
warnings.filterwarnings('ignore')


# Model

### define features extraction function

In [187]:
def mfcc_feature_extractor(audio,sampleRate):
    mfccsFeatures = librosa.feature.mfcc(y=audio, sr=sampleRate, n_mfcc=40)
    mfccsScaledFeatures = np.mean(mfccsFeatures.T,axis=0)
    
    return mfccsScaledFeatures

In [188]:
def contrast_feature_extractor(audio,sampleRate):
    stft = np.abs(librosa.stft(audio))
    contrast = librosa.feature.spectral_contrast(S=stft, sr=sampleRate) 
    contrastScaled = np.mean(contrast.T,axis=0)
    
    return contrastScaled

In [189]:
def tonnetz_feature_extractor(audio,sampleRate):
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(audio), sr=sampleRate)
    tonnetzScaled = np.mean(tonnetz.T,axis=0)
    
    return tonnetzScaled

In [190]:
def centroid_feature_extractor(audio,sampleRate):
    centroid = librosa.feature.spectral_centroid(y=audio,sr=sampleRate) 
    centroidScaled = np.mean(centroid.T,axis=0)
    
    return centroidScaled

In [191]:
def chroma_feature_extractor(audio,sampleRate):
    stft = np.abs(librosa.stft(audio))
    chroma = librosa.feature.chroma_stft(S=stft,sr=sampleRate)
    chromaScaled = np.mean(chroma.T,axis=0)
    
    return chromaScaled

In [192]:
def features_extractor(file):
    if file.lower().endswith(".m4a"):
        audio = AudioSegment.from_file(file, format="m4a")
        file_name, _ = os.path.splitext(os.path.basename(file))
        output_path = os.path.join(os.path.dirname(file), file_name + ".wav")
        audio.export(output_path, format="wav")
        os.remove(file)
        file = output_path

    features = []
    audio, sample_rate = librosa.load(file, res_type='kaiser_fast') 
    mfcc = mfcc_feature_extractor(audio, sample_rate)
    contrast = contrast_feature_extractor(audio, sample_rate)
    tonnetz = tonnetz_feature_extractor(audio, sample_rate)
    chroma = chroma_feature_extractor(audio, sample_rate)

    # Concatenate individual feature arrays
    concatenated_features = np.concatenate((mfcc, contrast, tonnetz, chroma), axis=0)
    features.append(concatenated_features)

    return features

### define add features function

In [193]:
def add_features(extractedFeatures, dirPath, label):
    for file in os.listdir(dirPath):
        filePath = os.path.join(dirPath, file)

        if filePath.lower().endswith(".m4a"):
            audio = AudioSegment.from_file(filePath, format="m4a")
            file_name, _ = os.path.splitext(os.path.basename(filePath))
            output_path = os.path.join(os.path.dirname(filePath), file_name + ".wav")
            audio.export(output_path, format="wav")
            os.remove(filePath)
            filePath = output_path

        audio, sampleRate = librosa.load(filePath, res_type='kaiser_fast') 
        mfcc = mfcc_feature_extractor(audio, sampleRate)
        contrast = contrast_feature_extractor(audio, sampleRate)
        tonnetz = tonnetz_feature_extractor(audio, sampleRate)
        chroma = chroma_feature_extractor(audio, sampleRate)

        extractedFeatures.append([mfcc, contrast, tonnetz, chroma, label])

### creating dictionary with dirPath and label

In [194]:
dict1 = {
    "ibrahim": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ibrahim",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/ibrahim",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/ibrahim",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/ibrahim",
    ],
    "ahmeda": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ahmeda",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/ahmeda",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/ahmeda",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/ahmeda",
    ],
    "omar": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/omar",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/omar",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/omar",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/omar",
    ],
    "hazem": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/hazem",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/hazem",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/hazem",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/hazem",
    ],
    "ahmedk": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ahmedk",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/ahmedk",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/ahmedk",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/ahmedk",
    ],
    "hassan": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/hassan",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/hassan",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/hassan",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/hassan",
    ],
    "mohannad": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/mohannad",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/mohannad",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/mohannad",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/mohannad",
    ],
    "other":[
        # "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other people", 
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/tamer", 
    ]
}


### creating data frame from our data

In [195]:
extractedFeatures = []
for label in dict1.keys():
    for link in dict1[label]:
        add_features(extractedFeatures, link, label)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/hazem\\Recording (13)_638396660156280910_wav.wav'

In [ ]:
data = pd.DataFrame(extractedFeatures, columns=['mfcc','contrast','tonnetz','chroma','class'])
data.to_csv('processing.csv')
print(data.shape)
data.describe().T

(318, 5)


,count,unique,top,freq
mfcc,318,318,"[-422.98352, 128.33788, 39.48789, 31.981009, 2...",1
contrast,318,318,"[22.726084348385594, 13.869069821878082, 16.11...",1
tonnetz,318,318,"[-0.007179744528078988, -0.005671084393048673,...",1
chroma,318,318,"[0.5809576, 0.59106517, 0.6385473, 0.599889, 0...",1
class,318,8,ahmedk,60


### splitting the data into features and target

In [ ]:
features = data.iloc[:,0:-1]
target = data['class']
target.unique()

array(['ibrahim', 'ahmeda', 'omar', 'hazem', 'ahmedk', 'hassan',
       'mohannad', 'other'], dtype=object)

In [ ]:
features = features.values.tolist()
for i in range(len(features)):
    features[i] = np.concatenate((features[i][0],features[i][1],features[i][2],features[i][3]))

### encoding the target

In [ ]:
encoder = LabelEncoder()
target = encoder.fit_transform(target)

In [ ]:
# # Assuming 'target' is your class labels
# encoder = LabelEncoder()
# target_encoded = encoder.fit_transform(target)

# # Display the mapping between original classes and encoded values
# class_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
# print("Class encoding mapping:")
# print(class_mapping)

### splitting the data into train and test

In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(features, target, test_size=0.25, random_state=29)

In [ ]:
for i in range(0,8,1):
  print(i,"  ", len(target[target==i]))
  # {'ahmeda': 0, 'ahmedk': 1, 'hassan': 2, 'hazem': 3, 'ibrahim': 4, 'mohannad': 5, 'omar': 6, 'other': 7}

0    34
1    60
2    32
3    50
4    47
5    33
6    43
7    19


### creating SVM model

In [ ]:
# classifier = SVC(kernel='linear',decision_function_shape="ovo" ,probability=True) 
# classifier.fit(xTrain,yTrain)
# # 99

SVC(decision_function_shape='ovo', kernel='linear', probability=True)

### creating XGBoost model

In [ ]:
classifier = XGBClassifier(objective='multi:softmax', num_class=8, eval_metric='mlogloss')
classifier.fit(xTrain, yTrain)
# 98

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_class=8, num_parallel_tree=None, ...)

### creating RandomForest model

In [ ]:
# classifier = RandomForestClassifier(n_estimators=100, random_state=42)  
# classifier.fit(xTrain, yTrain)
# # 97

### predicting xTest

In [ ]:
predX = classifier.predict(xTest)

### making classification report to see the result of the predicton 

In [ ]:
print(classification_report(yTest, predX))

train_predictions = classifier.predict(xTrain)
test_predictions = classifier.predict(xTest)

train_accuracy = accuracy_score(yTrain, train_predictions)
test_accuracy = accuracy_score(yTest, test_predictions)

print(train_accuracy, "  ", test_accuracy)

              precision    recall  f1-score   support

           0       1.00      0.86      0.92         7
           1       1.00      1.00      1.00        18
           2       0.67      1.00      0.80         6
           3       0.91      1.00      0.95        10
           4       1.00      0.93      0.97        15
           5       1.00      0.71      0.83         7
           6       1.00      1.00      1.00        10
           7       1.00      1.00      1.00         7

    accuracy                           0.95        80
   macro avg       0.95      0.94      0.93        80
weighted avg       0.96      0.95      0.95        80

1.0    0.95


In [ ]:
def prediction(pred):   
    # {'ahmeda': 0, 'ahmedk': 1, 'hassan': 2, 'hazem': 3, 'ibrahim': 4, 'mohannad': 5, 'omar': 6, 'other': 7}
    if pred == 0:
        return ("ahmeda")
    elif pred == 1:
        return ("ahmedk")
    elif pred == 2:
        return ("hassan")
    elif pred == 3:
        return ("hazem")
    elif pred == 4:
        return ("ibrahim")
    elif pred == 5:
        return ("mohannad")
    elif pred == 6:
        return ("omar")
    elif pred == 7:
        return ("other")

In [ ]:
def my_predict(features):
    probabilities = classifier.predict_proba(features)
    # print(probabilities)

    for i in range(len(features)):
        class_pred_index = probabilities[i].argmax()  
        class_pred = probabilities[i, class_pred_index]  
        threshold = 0.75
        
        if class_pred > threshold:
            return class_pred_index
        else:
            return 7

In [ ]:
for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/mohannad")[0:5]:
  testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/mohannad/{file}") 
  print(classifier.predict_proba(testFeatures))
  print("1: ",prediction(classifier.predict(testFeatures)))
  print("2: ",prediction(my_predict(testFeatures))) 

[[0.00114198 0.00134478 0.01020195 0.00112814 0.00280056 0.98009014
  0.00204635 0.00124609]]
1:  mohannad
2:  mohannad
[[0.00479955 0.00227053 0.05147755 0.00251426 0.00605994 0.9280978
  0.002346   0.00243433]]
1:  mohannad
2:  mohannad
[[0.01159821 0.00548679 0.5083017  0.00608004 0.0119465  0.4445827
  0.00612149 0.0058826 ]]
1:  hassan
2:  other
[[0.00341579 0.00161591 0.00163592 0.00186437 0.00431279 0.9836199
  0.00180284 0.00173248]]
1:  mohannad
2:  mohannad
[[0.00140807 0.00143307 0.00262249 0.00179454 0.00312024 0.9866044
  0.0014807  0.00153644]]
1:  mohannad
2:  mohannad


In [ ]:
for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/omar")[0:5]:
  testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/omar/{file}") 
  print(classifier.predict_proba(testFeatures))
  print("1: ",prediction(classifier.predict(testFeatures)))
  print("2: ",prediction(my_predict(testFeatures))) 

[[0.17547925 0.03591319 0.02009614 0.18441297 0.10902479 0.05922521
  0.3894359  0.02641259]]
1:  omar
2:  other
[[7.6900708e-04 1.9386989e-03 6.5866800e-04 1.2396399e-03 4.6947617e-03
  1.7529323e-03 9.8798847e-01 9.5772865e-04]]
1:  omar
2:  omar
[[7.7221508e-04 1.1721643e-03 6.6141569e-04 1.1746923e-03 2.1379651e-03
  1.1289258e-03 9.9210995e-01 8.4261846e-04]]
1:  omar
2:  omar
[[0.00121602 0.0015953  0.00104155 0.00165171 0.00149541 0.00150571
  0.9901675  0.00132689]]
1:  omar
2:  omar
[[1.3472161e-03 1.9101277e-03 5.3508376e-04 1.3678471e-03 5.5898582e-03
  1.0669188e-03 9.8750126e-01 6.8167638e-04]]
1:  omar
2:  omar


In [ ]:
# for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other people")[0:5]:
#   testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other people/{file}") 
#   print(classifier.predict_proba(testFeatures))
#   print("1: ",prediction(classifier.predict(testFeatures)))
#   print("2: ",prediction(my_predict(testFeatures))) 

[[0.05008704 0.12929615 0.04290043 0.36762446 0.10270782 0.05363643
  0.1924097  0.06133802]]
1:  hazem
2:  other
[[0.00737716 0.08635578 0.02640682 0.02331728 0.02084895 0.0060109
  0.8224764  0.00720672]]
1:  omar
2:  omar
[[0.00250406 0.00570408 0.00214477 0.00985747 0.00797931 0.00262586
  0.96611786 0.00306654]]
1:  omar
2:  omar
[[0.00926832 0.14521995 0.04829866 0.06802678 0.02363873 0.01642456
  0.6790096  0.01011332]]
1:  omar
2:  other
[[0.02931475 0.06677706 0.02510859 0.5949692  0.0844079  0.02639986
  0.14103526 0.0319874 ]]
1:  hazem
2:  other


In [ ]:
testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/dataset/recorded/1.wav") 
print(classifier.predict_proba(testFeatures))
print("1: ",prediction(classifier.predict(testFeatures)))
print("2: ",prediction(my_predict(testFeatures))) 

[[0.00162446 0.01523876 0.0015393  0.00698869 0.05553918 0.00226984
  0.9150272  0.00177256]]
1:  omar
2:  omar


In [ ]:
for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/tamer")[0:5]:
  testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/tamer/{file}") 
  print(classifier.predict_proba(testFeatures))
  print("1: ",prediction(classifier.predict(testFeatures)))
  print("2: ",prediction(my_predict(testFeatures))) 

[[0.00488285 0.00585894 0.0031662  0.00632452 0.00250653 0.00219525
  0.00263984 0.9724258 ]]
1:  other
2:  other
[[0.00488595 0.00379367 0.00194523 0.00780148 0.00406258 0.00207918
  0.00238824 0.97304374]]
1:  other
2:  other
[[0.00227124 0.00331717 0.00194536 0.00659058 0.00830087 0.00207931
  0.00238839 0.97310704]]
1:  other
2:  other
[[0.00224915 0.00754012 0.00192643 0.01417862 0.00606206 0.00203736
  0.00236516 0.96364117]]
1:  other
2:  other
[[0.00224984 0.00419257 0.00192702 0.00460121 0.01556905 0.00218374
  0.00534051 0.96393603]]
1:  other
2:  other


In [ ]:
pickle_out = open("processing.pkl", "wb")
pickle.dump(classifier, pickle_out)
pickle_out.close()